In [11]:
import numpy as np

class CalLoss:
    def __init__(self, y, y_pred):
      if y.shape != y_pred.shape:
            raise ValueError(f"Shape mismatch: y shape is {y.shape}, y_predicted shape is {y_predicted.shape}")
      self.y = y
      self.y_predicted = y_pred


class CrossEntropy(CalLoss):

    def give_celoss(self):
        return -np.mean(self.y * np.log(self.y_predicted))

    def Give_cegrad(self):
        grad = -self.y/(self.y_predicted)
        return grad

class SquaredError(CalLoss):

    def give_seloss(self):
        return np.mean((self.y - self.y_predicted) ** 2)

    def Give_segrad(self):
        res = -2*(self.y - self.y_predicted)
        return res

class callloss(CalLoss):
    def __init__(self, loss_function, y, y_pred):
        self.loss_function = loss_function.lower()
        super().__init__(y, y_pred)

    def give_loss(self):
        if self.loss_function == 'ce':
            return CrossEntropy(self.y, self.y_predicted).give_celoss()
        elif self.loss_function == 'se':
            return SquaredError(self.y, self.y_predicted).give_seloss()
        else:
            raise ValueError(f"Unknown loss function: {self.loss_function}")

    def give_gradloss(self):
        if self.loss_function == 'ce':
            return CrossEntropy(self.y, self.y_predicted).Give_cegrad()
        elif self.loss_function == 'se':
            return SquaredError(self.y, self.y_predicted).Give_segrad()
        else:
            raise ValueError(f"Unknown loss function: {self.loss_function}")

In [15]:
y = np.array([[0, 1, 0,],
             [1, 0, 0]])
y_predicted = np.array([[0.2, 0.7, 0.1],
                        [0.8, 0.1, 0.1]])

In [16]:
x = callloss('ce',y, y_predicted)
x.give_gradloss()

array([[ 0.        , -1.42857143,  0.        ],
       [-1.25      ,  0.        ,  0.        ]])

In [17]:
class CalculateAllLoss:
  def __init__(self, X_train, y_predicted,network, y_true, primary_loss, weight_decay=0, regularisation_fn=None):
    self.y_predicted = y_predicted
    self.y_true = y_true
    self.network = network
    self.X_train = X_train
    self.loss_value = primary_loss
    self.weight_decay = weight_decay
    self.regularisation_fn= regularisation_fn

class OverallLoss(CalculateAllLoss):

  def calc_total_loss(self):
    total_loss = self.loss_value  # Compute primary loss

    if self.weight_decay > 0 and self.regularisation_fn:
        regularized_val = ApplyReg(self.regularisation_fn, layer_dimension, self.network).do_reg()
        print(f"Reg value: {regularized_val}")
        total_loss += self.weight_decay * regularized_val # Add regularization term if applicable  # Compute total loss

    return total_loss


class AccuracyLoss(CalculateAllLoss):

  def calc_accuracy_loss(self):

    total_loss = self.overall_loss()

    # Ensure dimensions match between input and labels
    assert self.X_train.shape[1] == self.y_true.shape[1], "Mismatch in batch size between inputs and labels"

    # Compute accuracy by comparing predicted vs actual labels
    batch_size = self.X_train.shape[1]  # Number of examples
    correct_predictions = np.sum(np.argmax(self.y_predicted, axis=0) == np.argmax(self.y_true, axis=0))
    accuracy = correct_predictions / batch_size  # Compute accuracy as a fraction

    return accuracy, total_loss

